In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date

In [2]:
import plot_utils

In [3]:
working_dir = "/om2/user/ckauf/.result_caching/neural_nlp.models.wrapper.core.ActivationsExtractorHelper._from_sentences_stored"

In [4]:
model_identifier = 'gpt2-xl'
emb_context = 'Passage'
split_coord = "Sentence"
testonperturbed = True
randomnouns = False

In [5]:
# assimilate names between surpisals and brainscores

CONDMAP = {
    'Original': f'original',
    #
    'Scr1': f'scrambled1',  # different for teston, there teston:scr1
    'Scr3': f'scrambled3',
    'Scr5': f'scrambled5',
    'Scr7': f'scrambled7',
    'backward': f'backward',
    'lowPMI': f'lowpmi',
    'lowPMI-random': f'lowpmi-random',
    #
    'nouns': f'nouns',
    #'randomnouns': f'random-nouns',
    'nounsverbs': f'nounsverbs',
    'nounsverbsadj': f'nounsverbsadj',
    'contentwords': f'contentwords',
    'functionwords': f'functionwords',
    #
    'sentenceshuffle_passage': f'sent_passage',
    'sentenceshuffle_topic': f'sent_topic',
    'sentenceshuffle_random': f'sent_random',
    #
    'random-wl': f'random-wl'
}

In [6]:
def get_layers(model_identifier, which_layer="last"):
    """
    input: model_identifier of model of which we want to find the layers
    output: np.array of all unique layer identifiers, ordered by position
    """
    for ind,filename in enumerate(os.listdir(working_dir)):
        if "identifier=" + model_identifier in filename:
            file = os.path.join(working_dir,filename)
            with open(file, 'rb') as f:
                result = pickle.load(f)
            result = result['data']
            layer_list = np.unique(result.layer)
            #order double-digit layers at end of list
            double_digits = [elm for elm in layer_list if 'encoder.h.' in elm and len(elm.split('.h.')[-1]) > 1]
            layers = [e for e in layer_list if e not in double_digits] + double_digits
            
            if which_layer=="last":
                final_layer = layers[-1]
                return final_layer
            else:
                return layers
            break

In [7]:
def get_passage_identifier(filename):
    """
    get passage identifier to be used as key for the dictionary.
    important step: fill the identifier with 0s for single-digit passage numbers
    """
    passage = filename.split("-")[-1].split(".")[0]
    number = passage.split("sentences")[-1]
    if len(number) == 1:
        passage_identifier = passage[:-1] + number.zfill(2)
    else:
        passage_identifier = passage
    return passage_identifier

In [8]:
from tqdm.notebook import tqdm

def get_dictionary(model_identifier, emb_context=None):
    """
    input: model identifier
    output: populated model dictionary with data of just the selected layer (default: last_layer)
    dictionary structure: passage_identifier --> condition identifier --> data
    """
    
    model_dictionary = model_identifier + "_dict"
    model_dictionary = {}
    
    #look at last layer by default
    layer_identifier = get_layers(model_identifier, which_layer="last")

    print("This is the layer I'm looking at: ", layer_identifier)
    print(f"This is the emb_context={emb_context}")
    
    for cond in tqdm(CONDMAP.keys()):
        print(cond)
        for filename in tqdm(os.listdir(working_dir)):
            if f"identifier={model_identifier},stimuli_identifier=Pereira2018-{cond}-lasttoken,emb_context={emb_context}" in filename:
                passage_identifier = get_passage_identifier(filename)

                file = os.path.join(working_dir,filename)
                with open(file, 'rb') as f:
                    out = pickle.load(f)
                result = out['data']
                data = result[{"neuroid": [layer == layer_identifier for layer in result["layer"].values]}]

                if not cond in model_dictionary:
                    model_dictionary[cond] = {}
                model_dictionary[cond][passage_identifier] = data
    
    return model_dictionary

In [ ]:
model_activations_dictionary = get_dictionary(model_identifier=model_identifier, emb_context=emb_context)

/om2/user/ckauf/anaconda39/envs/perturbed3.8/lib/python3.8/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


This is the layer I'm looking at:  encoder.h.47
This is the emb_context=Passage


  0%|          | 0/17 [00:00<?, ?it/s]

Original


  0%|          | 0/63407 [00:00<?, ?it/s]

...

  0%|          | 0/63407 [00:00<?, ?it/s]

sentenceshuffle_random


  0%|          | 0/63407 [00:00<?, ?it/s]

random-wl


  0%|          | 0/63407 [00:00<?, ?it/s]

In [ ]:
assert model_activations_dictionary["Original"].keys() == model_activations_dictionary["Scr1"].keys()
print("Same")

In [ ]:
model_activations_dictionary["Original"].keys()

In [ ]:
print(len(model_activations_dictionary["Original"]))
print(len(model_activations_dictionary["Original"]["243sentences01"]))
print(len(model_activations_dictionary["Original"]["243sentences01"][0]))

In [ ]:
def flatten_array(liste):
    liste_flatten = [item for sublist in liste for item in sublist]
    return liste_flatten

In [ ]:
model_activations_dictionary.keys()

In [ ]:
flatten_dict = {}
for cond in tqdm(CONDMAP.keys()):
    flatten_array_passage = flatten_array(model_activations_dictionary[f"{cond}"].values())
    flatten_array_activations = np.asarray(flatten_array_passage).flatten()
    flatten_dict[cond] = flatten_array_activations
flatten_dict.keys()
assert len(flatten_dict["Original"]) == (627 * 1600)
print("Asserted!")

In [ ]:
from scipy import stats
def get_correlations_df_dict(flatten_dict):
    
    correlations_df_dict = {}
    for cond in tqdm(CONDMAP.keys()):
        orig_column = flatten_dict['Original']
        correlation = stats.spearmanr(orig_column,flatten_dict[cond])[0]
        correlations_df_dict[cond] = correlation
    
    return correlations_df_dict

In [ ]:
correlations_df_dict = get_correlations_df_dict(flatten_dict)

In [ ]:
correlations_df_dict

# Plot

In [ ]:
CAT2COND , COND2CAT = plot_utils.get_conditions(testonperturbed=False)

In [ ]:
# super ugly - redo!
correlations_df = pd.DataFrame.from_dict([correlations_df_dict]).T.reset_index()
correlations_df.columns = ["condition", "Spearman correlation"]
#Rename conditions
correlations_df['condition'] = correlations_df['condition'].map(CONDMAP)
correlations_df['condition'] = correlations_df['condition'].replace(
        {'sent_random': 'sentenceshuffle_random',
    'sent_passage': 'sentenceshuffle_passage',
    'sent_topic': 'sentenceshuffle_topic'})
correlations_df['category'] = correlations_df['condition'].map(COND2CAT)
#Rename conditions according to new names
correlations_df['condition'] = correlations_df['condition'].replace(
        {'sentenceshuffle_random': 'sent_random',
    'sentenceshuffle_passage': 'sent_passage',
    'sentenceshuffle_topic': 'sent_topic'})
correlations_df['condition'] = correlations_df['condition'].map(plot_utils.COND2LABEL)
correlations_df

In [ ]:
correlations_df.to_csv("last_layer_correlations.csv", index=False)

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['svg.fonttype'] = 'none'

custom_params = {"axes.spines.right": False,
                 "axes.spines.top": False,
                 'ytick.left': True,
                 'xtick.bottom': True,
                'grid.linestyle': "", #gets rid of horizontal lines
                 # set tick width
                'xtick.major.size' : 20,
                'xtick.major.width': 4,
                'xtick.minor.size' : 10,
                'xtick.minor.width' : 2,
                'ytick.major.size' : 20,
                'ytick.major.width': 4,
                'ytick.minor.size' : 10,
                'ytick.minor.width' : 2
                }

sns.set_theme(font_scale=2.5, style="white", rc=custom_params)

In [ ]:
def func(string):
    return "\n".join(string.split("From"))

correlations_df['condition'] = correlations_df['condition'].apply(func)

In [ ]:
#get proportions for subplot width
categories = CAT2COND.keys()
props = [len(correlations_df.loc[correlations_df["category"] == cat])/len(correlations_df) for cat in categories]

palette = sns.color_palette('magma')

g = sns.FacetGrid(correlations_df, col="category", sharex=False,
                  gridspec_kws={"width_ratios": props},
                  height=10, aspect=0.5)

g.map(sns.lineplot, "condition", "Spearman correlation", alpha=0.8, markersize=14, marker="o", linewidth=2.5, color=palette[0])
g.set_xlabels("")
g.set_ylabels("Spearman's rho")
g.set(ylim=(0, 1.05))
g.set_titles(col_template="{col_name}", pad=0.1)
g.set_xticklabels(rotation=80)

g.fig.tight_layout()

plt.savefig(f'results_paper/correlations2brainscore/last-layer-correlations.png', bbox_inches='tight', dpi=300)
plt.savefig(f'results_paper/correlations2brainscore/last-layer-correlations.svg', bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
break

In [ ]:
def plot_correlations_lineplot(model_identifier, layers, conditions, correlations_dict,scr_perturb=None,
                              emb_context=None):
    fig, ax = plt.subplots()
    line_colors = sns.color_palette("rocket") + sns.color_palette("GnBu_d") + sns.color_palette("PRGn", 10) + sns.color_palette("YlOrBr", 10)
    
    counter = 0
    for key,value in correlations_dict.items():
        ax.plot(conditions,value, '-o',color=line_colors[counter])
        counter += 1

    ax.set_title(f'{model_identifier} | Layer model activation correlation with model activations for original sentence across conditions')
    ax.legend(layers)
    ax.yaxis.set_label_text('Spearman rho')
    #ax.yaxis.set_label_text('Pearson p')
    ax.legend(layers, bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.set_ylim([0, 1])
    plt.xticks(rotation= 90)
    
    if not os.path.isdir("fig"):
        os.mkdir("fig")
    savename = f'fig/{date.today()}_model-activation-correlation_{model_identifier}_{scr_perturb}.png'
    #plt.savefig(savename, bbox_inches='tight', dpi=240)

In [ ]:
emb_context="Passage"

In [ ]:
layers, conditions, corr_dict = get_correlations_df_dict("gpt2-xl",categories_dict, scr_perturb="perturb", emb_context=emb_context)
plot_correlations_lineplot("gpt2-xl", layers, conditions, corr_dict, scr_perturb="perturb", emb_context=emb_context)

In [ ]:
layers, conditions, corr_dict = get_correlations_df_dict("gpt2-xl",categories_dict, scr_perturb="scrambled", emb_context=emb_context)
plot_correlations_lineplot("gpt2-xl", layers, conditions, corr_dict, scr_perturb="scrambled", emb_context=emb_context)

In [ ]:
# layers, conditions, corr_dict = get_correlations_df_dict("gpt2-xl",categories_dict, scr_perturb="perturb", emb_context="Passage")
# plot_correlations_lineplot("gpt2-xl", layers, conditions, corr_dict, scr_perturb="perturb")